# 네이버 뉴스 크롤링

In [1]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup as bs
import soupsieve as sv

from urllib.request import Request, urlopen
from urllib.error import HTTPError, URLError
from urllib.parse import quote_plus

## session 세팅

In [2]:
session = requests.session()

assert_status_hook = lambda response, *args, **kwargs: response.raise_for_status()
session.hooks["response"] = [assert_status_hook]

retry_config = {
    "total": 10,
    "status_forcelist": [413, 429, 500, 502, 503, 504],
    "method_whitelist": ["GET", "POST"],
    "backoff_factor": 2,
    }
retry_strategy = Retry(**retry_config)
adapter = HTTPAdapter(max_retries=retry_strategy)

session.mount("http://", adapter)
session.mount("https://", adapter)

C:\Users\chlje\AppData\Local\Temp/ipykernel_34348/2081969212.py:12: DeprecationWarning: Using 'method_whitelist' with Retry is deprecated and will be removed in v2.0. Use 'allowed_methods' instead
  retry_strategy = Retry(**retry_config)


## Crawl news data

In [3]:
query = '삼성전자'

In [4]:
quote_plus(query) # url 형식으로 encode

'%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90'

In [5]:

# 검색어, 날짜 집어넣으면 뉴스 검색 결과 내보내주는 함수 
# 참고: start_idx는 페이지마다 몇 번째 뉴스인가 라는 값인데, 페이지 넘어갈때마다 10씩 늘어남. 
#       그러니까 1, 11, 21, ... 으로 늘려가며 페이지 쭉~ 긁으면 됨. 

def naver_news_search_url(query, start_idx, start_date, end_date=None, sort_key='recent_asc'):
    sort_key = sort_key.lower()
    assert sort_key in ['recent_asc', 'relevance']
    sort_d = {
        'recent_asc': '1',
        'relevance': '0',
    }

    encoded_query = quote_plus(query)

    start_date = str(start_date)
    start_date_dot = start_date[:4] + '.' + start_date[4:6] + '.' + start_date[6:]

    if end_date:
        end_date = str(end_date)
        end_date_dot = end_date[:4] + '.' + end_date[4:6] + '.' + end_date[6:]
    else:
        end_date_dot = start_date_dot
        end_date = start_date

    # url = f'https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort={sort_d[sort_key]}&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&start={start_idx}'
    # url = f'https://m.search.naver.com/search.naver?where=m_news&sm=mtb_pge&query={query}&sort={sort_d[sort_key]}&photo=0&field=0&pd=0&ds={start_date_dot}&de={end_date_dot}&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:all&start={start_idx}'
    url = f'https://m.search.naver.com/search.naver?where=m_news&sm=mtb_pge&query={encoded_query}&sort={sort_d[sort_key]}&photo=0&field=0&pd=3&ds={start_date_dot}&de={end_date_dot}&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from{start_date}to{end_date}&start={start_idx}'
    
    return url

In [6]:
start_date = 20220530 # 크롤링할 날짜

In [36]:
u = naver_news_search_url(query=query, start_idx=1, start_date=start_date) # 이 url 페이지꺼를 긁어오는거임. 
u

'https://m.search.naver.com/search.naver?where=m_news&sm=mtb_pge&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sort=1&photo=0&field=0&pd=3&ds=2022.05.30&de=2022.05.30&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from20220530to20220530&start=1'

In [37]:
headers = {
            "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
            "accept-encoding": "gzip, deflate, br",
            "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
            "cache-control": "max-age=0",
            "referer": u,
            "sec-ch-ua": '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": "Windows",
            "sec-fetch-dest": "document",
            "sec-fetch-mode": "navigate",
            "sec-fetch-site": "same-origin",
            "sec-fetch-user": "?1",
            "upgrade-insecure-requests": "1",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36",
            }

In [38]:

r = session.get(u, headers=headers)
# r = requests.get(u, headers=headers)

In [39]:
r # 정상 연결 확인할 수 있음. 

<Response [200]>

In [40]:
b = bs(r.content, 'html.parser') # beautiful soup object로 파싱해서

In [41]:
news_urls = b.find_all('div', {'class': 'news_dsc'}) # 기사 url만 가져옴. 
news_urls = [n.find('a').attrs['href'] for n in news_urls] 


In [46]:
news_urls

['http://www.jejumaeil.net/news/articleView.html?idxno=316038',
 'https://m.asiatime.co.kr/article/20220530500404',
 'http://www.itdaily.kr/news/articleView.html?idxno=208334',
 'https://byline.network/?p=9004111222475733',
 'http://m.betanews.net:8080/article/1337978.html',
 'http://www.ziksir.com/news/articleView.html?idxno=25627',
 'http://www.econotelling.com/news/articleView.html?idxno=6826',
 'http://www.s-journal.co.kr/news/articleView.html?idxno=5624',
 'http://www.meconomynews.com/news/articleView.html?idxno=67149',
 'https://m.news.naver.com/read?mode=LSD&mid=sec&sid1=101&oid=032&aid=0003150249',
 'http://www.ziksir.com/news/articleView.html?idxno=25617',
 'https://www.natv.go.kr/natv/news/newsView.do?newsId=503939',
 'https://m.news.naver.com/read?mode=LSD&mid=sec&sid1=105&oid=015&aid=0004705742',
 'https://m.news.naver.com/read?mode=LSD&mid=sec&sid1=101&oid=022&aid=0003700698',
 'https://www.dailyimpact.co.kr/news/articleView.html?idxno=80078']

In [42]:
len(news_urls)

15

In [43]:
companies = b.find_all('div', {'class': 'info_group'})
companies = [c.find('a').get_text() for c in companies]
companies

['제주매일',
 '아시아타임즈',
 '아이티데일리',
 '바이라인네트워크',
 '베타뉴스',
 '직썰',
 '이코노텔링',
 'S-저널',
 '시장경제신문',
 '경향신문',
 '직썰',
 '국회방송',
 '한국경제언론사 선정',
 '세계일보언론사 선정',
 '데일리임팩트']

In [45]:
titles = b.find_all('div', {'class': 'tit'})
titles = [t.get_text() for t in titles]
titles

['삼성을 유치한 평택의 기적을 보며',
 "[마감] 원숭이두창에 소아급성간염까지...차백신연구소 등 '급등'",
 '에이수스, 올 1분기 커머셜 노트북 시장서 시장점유율 1위 달성',
 '삼성 이재용과 인텔 팻 겔싱어는 왜 만났을까',
 '[주요공시]엘브이엠씨-신송홀딩스-삼성전자-한화생명-삼보판자',
 "이재용 '리더십' 진일보, 인텔 CEO와 '韓美 반도체 협력' 속도",
 "이재용 - 팻 겔싱어 '韓美 반도체 정상회의'",
 "세계 1·2위 만남… 이재용, 팻 겔싱어 인텔 CEO와 '반도체 협력' 본격 시동",
 "[시경Today] 롯데아울렛, '서프라이스 위크'... 신세계사이먼, '쇼핑 카니발' 外",
 '‘반도체 라이벌’ 삼성 이재용·인텔 겔싱어 회동',
 '이재용, 겔싱어 인텔 CEO와 반도체 협력방안 논의',
 '원조 친노 이광재 역전승 vs 기사회생 김진태 굳히기',
 "노트북도 한국 제쳤다…삼성·LG 누르고 '첫 1위' 이변",
 '이재용, 인텔 겔싱어 CEO와 회동… 반도체 협력 강화하나',
 "한·일 관계 '봄' 오나…양국 기업인 '의기투합'"]

In [16]:
naver_news_str = 'm.news.naver.com'
news_urls = [u for u in news_urls if naver_news_str in u] # 그 중 네이버 뉴스 형식으로 되어있는거만 모음. 

In [17]:
news_urls # 그 페이지에서 남은 결과 

['https://m.news.naver.com/read?mode=LSD&mid=sec&sid1=101&oid=032&aid=0003150249',
 'https://m.news.naver.com/read?mode=LSD&mid=sec&sid1=101&oid=022&aid=0003700698',
 'https://m.news.naver.com/read?mode=LSD&mid=sec&sid1=105&oid=015&aid=0004705742']

재영 여기서부터 해줄꺼:

- 각 기사 url 타고 들어가서 [회사명, 제목, 신문사, 기자, 입력날짜, 내용본문] 이렇게 뽑아주는 크롤링 함수 만들어주면 됨.
- output 형태는 굳이 pandas로 할 필요 없으며, list of dictionaries 가 나을 것 같음. 
    - 예시: [
        {
            '회사': '삼성전자',
            '제목': '삼성전자 이재용이...',
            '신문사': '조선일보',
            '기자': '윤재영',
            '입력날짜': '2022.05.30 오후 5:57',
            '내용본문': '이재용 회장이..',
            },
        {
            '회사': '삼성전자',
            '제목': '삼성전자 반도체가...',
            '신문사': '조선일보',
            '기자': '최재필',
            '입력날짜': '2022.05.30 오후 6:27',
            '내용본문': '반도체 시장이..',
            },
    ]

- 너가 함수 만드는 동안 나도 이거 전종목 크롤링 돌리는 함수 만들고 url 모아두고 있을께 ㅇㅇ 

In [18]:
def article_crawling(url_link):
  url=url_link
  r=session.get(url, headers=headers)
  b=bs(r.content,'html.parser')
  news_head = b.find('h2', {'class': 'media_end_head_headline'}) # 기사 제목만 가져옴. 
  headline=news_head.get_text()
  news_text = b.find('div', {'id': 'dic_area'}) # 기사 본문만 가져옴. 
  text=news_text.get_text()
  news_section = b.find('em', {'class': 'media_end_categorize_item'}) # 기사 섹션만 가져옴.
  section=news_section.get_text()
  news_writer = b.find('em', {'class': 'media_end_head_journalist_name'}) # 기자이름만 가져옴.
  writer=news_writer.get_text()
  news_link = b.find('a', {'class': "media_end_head_origin_link"} ).attrs['href'] # 기사 링크만 가져옴.
  link=news_link
  news_date = b.find('span', {'class': 'media_end_head_info_datestamp_time _ARTICLE_DATE_TIME'}) # 기사 날짜만 가져옴.
  date=news_date.get_text()
  result={'headline':headline,'date':date,'writer':writer,'section':section,'link':link,'text':text}
  return result


  

In [21]:
url= "https://n.news.naver.com/mnews/article/032/0003150249?sid=101"

r = session.get(url, headers=headers)

In [22]:
b = bs(r.content, 'html.parser') # beautiful soup object로 파싱해서

In [23]:
news_head = b.find('h2', {'class': 'media_end_head_headline'}) # 기사 제목만 가져옴. 
headline=news_head.get_text()
headline

'‘반도체 라이벌’ 삼성 이재용·인텔 겔싱어 회동'

In [24]:
news_text = b.find('div', {'id': 'dic_area'}) # 기사 본문만 가져옴. 
text=news_text.get_text()
text

'\n한·미 정상 평택공장 방문 열흘 만양사 경영진도 참여, 릴레이 회의파운드리 등 협력분야 확대될 듯이재용 삼성전자 부회장(사진)이 30일 팻 겔싱어 인텔 최고경영자(CEO)와 만나 협력방안을 논의했다. 조 바이든 미국 대통령과 윤석열 대통령이 지난 20일 삼성전자 평택 공장을 방문한 지 열흘 만에 반도체업계 1위와 2위 업체의 수장이 만나 반도체 분야 협력 방안에 대한 의견을 교환한 것이다.이날 삼성전자에 따르면 이 부회장은 방한 중인 겔싱어 CEO를 만나 차세대 메모리, 시스템반도체 설계(팹리스)·위탁생산(파운드리), PC와 모바일 등 다양한 분야에서의 협력 방안을 논의했다. 이 자리에는 이 부회장과 겔싱어 CEO 외에 두 업체 경영진도 참여했다. 삼성전자에서는 경계현 DS부문장, 노태문 MX사업부장, 이정배 메모리사업부장, 최시영 파운드리사업부장, 박용인 시스템LSI사업부장 등이 배석해 인텔의 임원들과 릴레이 회의를 한 것으로 전해졌다.삼성전자와 인텔은 반도체 시장에서 1, 2위를 다투는 ‘라이벌’ 관계다. 삼성전자는 지난해 반도체에서 823억달러의 매출을 올려 인텔(790억달러)을 제치고 다시 1위에 올랐다. 삼성전자가 인텔을 앞선 것은 2018년 이후 3년 만이다. 지난해 3월 인텔이 파운드리 시장 진출을 공식 선언하면서 앞으로 파운드리 분야에서 대만 TSMC와 삼성전자, 인텔의 ‘3파전’이 예상된다. 메모리 반도체 1위인 삼성전자는 2030년까지 파운드리 분야에서도 세계 1위로 올라서겠다는 계획을 밝힌 바 있다.두 업체가 손을 잡을 가능성도 있다. 겔싱어 CEO는 지난해 1월 실적 발표에서 “우리 포트폴리오를 고려할 때 특정 기술과 제품에 대한 외부 파운드리 사용은 더 늘려갈 것으로 예상된다”고 말했다. 반도체업계에서는 인텔이 주력 제품인 중앙처리장치(CPU)는 자체 생산하고, 나머지 칩셋 같은 제품은 삼성전자와 대만의 TSMC 등에 생산을 맡길 것이라는 해석이 나온다. 반도체업계 관계자는 “인텔이 글로벌 반도체 수급 불균형을 해소하기 위해서는 1

In [25]:
# !pip install pororo

In [26]:
news_section = b.find('em', {'class': 'media_end_categorize_item'}) # 기사 본문만 가져옴.
section=news_section.get_text()
section

'경제'

In [27]:
news_writer = b.find('em', {'class': 'media_end_head_journalist_name'}) # 기사 본문만 가져옴.
writer=news_writer.get_text()
writer

'이재덕 기자'

In [28]:
news_link = b.find('a', {'class': "media_end_head_origin_link"} ).attrs['href'] # 기사 본문만 가져옴.
link=news_link
link

'https://www.khan.co.kr/economy/industry-trade/article/202205302227025'

In [29]:
news_date = b.find('span', {'class': 'media_end_head_info_datestamp_time _ARTICLE_DATE_TIME'}) # 기사 본문만 가져옴.
date=news_date.get_text()
date

'2022.05.30. 오후 10:29'

In [26]:
# from pororo import Pororo

In [30]:
# sa = Pororo(task='sentiment', model='brainbert.base.ko.nsmc', lang='ko')
# sa(text, show_probs=True)

{'negative': 0.17410160601139069, 'positive': 0.8258984088897705}

In [31]:
# sa('아 우리 조 팀플 준비 하려면 지금 너무 시간이 없다. 상황이 너무 어렵다', show_probs=True)

{'negative': 0.9926801919937134, 'positive': 0.007319830823689699}